In [1]:
%load_ext autoreload
%autoreload 2
from Methods import sentenceSimilarityMethods,JobOffers,dataframeHelpers

In [2]:
import json
import os
import nltk
import re
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
from scipy import spatial

In [4]:
autoreload JobOffers,dataframeHelpers

In [5]:
jobsDf = dataframeHelpers.buildJobsDF()

In [6]:
skillsDf = dataframeHelpers.buildSkillsDF()


In [7]:
skillsDf

,CATEGORY,SKILL_CONTENT,SKILL_ID
0,Complex problem solving skills,Identifying complex problems and reviewing rel...,296
1,Resource management skills,Developed capacities used to allocate resource...,297
2,Management of Financial Resources,Determining how money will be spent to get the...,298
3,Management of Material Resources,Obtaining and seeing to the appropriate use of...,299
4,Management of Personnel Resources —,"Motivating, developing, and directing people a...",300
...,...,...,...
78,Transportation,Knowledge of principles and methods for moving...,387
79,Diplôme,Diploma equivalent,390
80,diplôme équivalent,Diploma or cfc,391
81,master,Master’s degree,392


In [8]:
embeddings = dataframeHelpers.getEmbeddingsWSkills(skillsDf, jobsDf)

In [9]:
jobsDf = dataframeHelpers.addEmbeddingsToJobs(jobsDf, embeddings)
skillsDf = dataframeHelpers.addEmbeddingsToSkills(skillsDf, embeddings)

In [10]:
skillsDf

,CATEGORY,SKILL_CONTENT,SKILL_ID,SKILL_SCORES
0,Complex problem solving skills,Identifying complex problems and reviewing rel...,296,"(tf.Tensor(0.057766527, shape=(), dtype=float3..."
1,Resource management skills,Developed capacities used to allocate resource...,297,"(tf.Tensor(0.079673186, shape=(), dtype=float3..."
2,Management of Financial Resources,Determining how money will be spent to get the...,298,"(tf.Tensor(0.029886357, shape=(), dtype=float3..."
3,Management of Material Resources,Obtaining and seeing to the appropriate use of...,299,"(tf.Tensor(-0.017312534, shape=(), dtype=float..."
4,Management of Personnel Resources —,"Motivating, developing, and directing people a...",300,"(tf.Tensor(-0.011685572, shape=(), dtype=float..."
...,...,...,...,...
78,Transportation,Knowledge of principles and methods for moving...,387,"(tf.Tensor(0.08552032, shape=(), dtype=float32..."
79,Diplôme,Diploma equivalent,390,"(tf.Tensor(-0.020993702, shape=(), dtype=float..."
80,diplôme équivalent,Diploma or cfc,391,"(tf.Tensor(0.052724365, shape=(), dtype=float3..."
81,master,Master’s degree,392,"(tf.Tensor(0.0395185, shape=(), dtype=float32)..."


In [11]:
skillsResultsDf = pd.DataFrame()

In [12]:
for label in ["JOB_ID", "JOB_SCORES"]:
    skillsResultsDf[label] = jobsDf[label]

In [13]:
skillsResultsDf = skillsResultsDf.merge(skillsDf, how = "cross")

In [14]:
skillsResultsDf = skillsResultsDf[["JOB_ID", "SKILL_ID", "JOB_SCORES", "SKILL_SCORES"]]

In [15]:
skillsResultsDf

,JOB_ID,SKILL_ID,JOB_SCORES,SKILL_SCORES
0,12220025,296,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(0.057766527, shape=(), dtype=float3..."
1,12220025,297,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(0.079673186, shape=(), dtype=float3..."
2,12220025,298,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(0.029886357, shape=(), dtype=float3..."
3,12220025,299,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(-0.017312534, shape=(), dtype=float..."
4,12220025,300,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(-0.011685572, shape=(), dtype=float..."
...,...,...,...,...
8295,13462067,387,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(0.08552032, shape=(), dtype=float32..."
8296,13462067,390,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(-0.020993702, shape=(), dtype=float..."
8297,13462067,391,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(0.052724365, shape=(), dtype=float3..."
8298,13462067,392,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(0.0395185, shape=(), dtype=float32)..."


In [16]:
def compareSentences(row):
    for sentence in skillsResultsDf["JOB_SCORES"]:
        return spatial.distance.cosine(row["JOB_SCORES"][sentence], row["SKILL_SCORES"])

In [17]:
def distCalc(row):
    distArray = []
    for entry in row["JOB_SCORES"] :
        dist = spatial.distance.cosine(entry, row["SKILL_SCORES"])
        distArray.append(dist)
    return distArray

In [18]:
skillsResultsDf["DISTANCES"] = skillsResultsDf.apply(distCalc, axis =1)

In [19]:
def minDistanceFinder(row) : 
    minDistance = min(row["DISTANCES"])
    return minDistance

In [20]:
def minDistanceSentence(row, jobs) : 
    minDistanceIdx = np.argmin(row["DISTANCES"])
    df = jobs[jobs["JOB_ID"] == row["JOB_ID"]]
    if df.empty:
        return ""
    content = df.iloc[0]["JOB_CONTENT"]
    return content[minDistanceIdx]

In [21]:
skillsResultsDf["LOWEST_DISTANCE"] = skillsResultsDf.apply(minDistanceFinder, axis =1)
skillsResultsDf["LOWEST_DISTANCE_SENTENCE"] = skillsResultsDf.apply(minDistanceSentence, args=[jobsDf], axis =1)

In [22]:
skillsResultsDf

,JOB_ID,SKILL_ID,JOB_SCORES,SKILL_SCORES,DISTANCES,LOWEST_DISTANCE,LOWEST_DISTANCE_SENTENCE
0,12220025,296,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(0.057766527, shape=(), dtype=float3...","[1.1267676651477814, 0.9312523379921913, 0.989...",0.829701,They are easy to handle and want to develop a...
1,12220025,297,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(0.079673186, shape=(), dtype=float3...","[1.054056640714407, 1.008940627798438, 1.07808...",0.857967,You get the opportunity to grow in the task
2,12220025,298,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(0.029886357, shape=(), dtype=float3...","[0.9942623521201313, 0.870350256562233, 0.9467...",0.783709,They want to gain experience in project plann...
3,12220025,299,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(-0.017312534, shape=(), dtype=float...","[1.0033741996157914, 0.8672861456871033, 1.042...",0.863978,Your chances You can expand your knowledge an...
4,12220025,300,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(-0.011685572, shape=(), dtype=float...","[0.9054757431149483, 0.8429013639688492, 0.901...",0.777023,They want to gain experience in project plann...
...,...,...,...,...,...,...,...
8295,13462067,387,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(0.08552032, shape=(), dtype=float32...","[0.9634885378181934, 0.9429045058786869, 1.099...",0.942905,150
8296,13462067,390,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(-0.020993702, shape=(), dtype=float...","[0.9652066752314568, 0.8018802851438522, 0.969...",0.751800,08
8297,13462067,391,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(0.052724365, shape=(), dtype=float3...","[0.8277485966682434, 0.857603907585144, 1.0255...",0.827749,Haas Gartenbau AGGarden and Landscaping EFZBer...
8298,13462067,392,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(0.0395185, shape=(), dtype=float32)...","[0.9429786913096905, 0.805667445063591, 0.9034...",0.805667,150


In [23]:
def bestMatchReturn(row, results, skillsDf, match, column_name) :
    bestMatch = results[results["JOB_ID"] == row["JOB_ID"]]
    minEntry = bestMatch[match].idxmin()
    rowMatch = bestMatch.loc[minEntry,:]  
    #return actsDf[actsDf["ACTIVITY_ID"] == rowMatch["ACTIVITY_ID"]]["ACTIVITY_ID"]
    content = skillsDf[skillsDf["SKILL_ID"] == rowMatch["SKILL_ID"]]                  
    return content[column_name].iloc[0]  

In [24]:
def bestNMatchReturn(row, results, skillsDf, match, column_name, num) :
    jobMatch = results[results["JOB_ID"] == row["JOB_ID"]]
    minEntries = jobMatch[match].argsort()[:num]
    rowMatch = jobMatch.iloc[minEntries,:]
    content = [skillsDf[skillsDf["SKILL_ID"] == x][column_name].iloc[0] for x in rowMatch["SKILL_ID"]]
    return content

In [25]:
def averageNSentences(row, n):
    return np.average(np.sort(row["DISTANCES"])[:n])

In [26]:
skillsResultsDf["AVERAGE_DISTANCE_3SENT"] = skillsResultsDf.apply(averageNSentences, args=[3], axis=1)

In [27]:
skillsResultsDf

,JOB_ID,SKILL_ID,JOB_SCORES,SKILL_SCORES,DISTANCES,LOWEST_DISTANCE,LOWEST_DISTANCE_SENTENCE,AVERAGE_DISTANCE_3SENT
0,12220025,296,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(0.057766527, shape=(), dtype=float3...","[1.1267676651477814, 0.9312523379921913, 0.989...",0.829701,They are easy to handle and want to develop a...,0.840660
1,12220025,297,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(0.079673186, shape=(), dtype=float3...","[1.054056640714407, 1.008940627798438, 1.07808...",0.857967,You get the opportunity to grow in the task,0.891139
2,12220025,298,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(0.029886357, shape=(), dtype=float3...","[0.9942623521201313, 0.870350256562233, 0.9467...",0.783709,They want to gain experience in project plann...,0.852414
3,12220025,299,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(-0.017312534, shape=(), dtype=float...","[1.0033741996157914, 0.8672861456871033, 1.042...",0.863978,Your chances You can expand your knowledge an...,0.874443
4,12220025,300,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(-0.011685572, shape=(), dtype=float...","[0.9054757431149483, 0.8429013639688492, 0.901...",0.777023,They want to gain experience in project plann...,0.806132
...,...,...,...,...,...,...,...,...
8295,13462067,387,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(0.08552032, shape=(), dtype=float32...","[0.9634885378181934, 0.9429045058786869, 1.099...",0.942905,150,0.967948
8296,13462067,390,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(-0.020993702, shape=(), dtype=float...","[0.9652066752314568, 0.8018802851438522, 0.969...",0.751800,08,0.831690
8297,13462067,391,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(0.052724365, shape=(), dtype=float3...","[0.8277485966682434, 0.857603907585144, 1.0255...",0.827749,Haas Gartenbau AGGarden and Landscaping EFZBer...,0.855176
8298,13462067,392,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(0.0395185, shape=(), dtype=float32)...","[0.9429786913096905, 0.805667445063591, 0.9034...",0.805667,150,0.861207


In [28]:
row = skillsResultsDf.loc[2]
row

JOB_ID                                                               12220025
SKILL_ID                                                                  298
JOB_SCORES                  ((tf.Tensor(-0.037420016, shape=(), dtype=floa...
SKILL_SCORES                (tf.Tensor(0.029886357, shape=(), dtype=float3...
DISTANCES                   [0.9942623521201313, 0.870350256562233, 0.9467...
LOWEST_DISTANCE                                                      0.783709
LOWEST_DISTANCE_SENTENCE     They want to gain experience in project plann...
AVERAGE_DISTANCE_3SENT                                               0.852414
Name: 2, dtype: object

In [29]:
jobMatch = skillsResultsDf[skillsResultsDf["JOB_ID"] == row["JOB_ID"]]
minEntries = jobMatch["LOWEST_DISTANCE"].argsort()[:10]

In [30]:
column_names = ["JOB_ID", "ISCO", "JOB_CONTENT", "TOP_SKILL_MATCHES", "TOP_SKILL_IDS", "BEST_SKILL_MATCH_3SENT", "BEST_SKILL_ID_3SENT", "BEST_SKILL_MATCH_1SENT", "BEST_SKILL_ID_1SENT", "LOWEST_DISTANCE", ]

In [31]:
topResultsDf = pd.DataFrame(columns = column_names)

In [32]:
for label in ["JOB_ID", "ISCO", "JOB_CONTENT", "ORIGINAL_CONTENT"]:
    topResultsDf[label] = jobsDf[label]

In [33]:
topResultsDf["LOWEST_DISTANCE"] = skillsResultsDf["LOWEST_DISTANCE"]
topResultsDf["AVERAGE_DISTANCE_3SENT"] = skillsResultsDf["AVERAGE_DISTANCE_3SENT"]

In [34]:
topResultsDf["JOB_CONTENT"] = jobsDf.apply(lambda x: " ".join(x["JOB_CONTENT"]), axis=1)

In [35]:
topResultsDf

,JOB_ID,ISCO,JOB_CONTENT,TOP_SKILL_MATCHES,TOP_SKILL_IDS,BEST_SKILL_MATCH_3SENT,BEST_SKILL_ID_3SENT,BEST_SKILL_MATCH_1SENT,BEST_SKILL_ID_1SENT,LOWEST_DISTANCE,ORIGINAL_CONTENT,AVERAGE_DISTANCE_3SENT
0,12220025,21420,We are a successful and independent engineerin...,NaN,NaN,NaN,NaN,NaN,NaN,0.829701,<p>Wir sind ein erfolgreiches und unabhängiges...,0.840660
1,10814567,21420,We are a renowned and successful engineering c...,NaN,NaN,NaN,NaN,NaN,NaN,0.857967,Wir sind eine renommierte und erfolgreiche Ing...,0.891139
2,12401674,21420,Bill Weyermann Partner AG has been providing e...,NaN,NaN,NaN,NaN,NaN,NaN,0.783709,Bill Weyermann Partner AG erbringt seit über\n...,0.852414
3,12473085,62100,Administration communale de ValdeRuzForstwarti...,NaN,NaN,NaN,NaN,NaN,NaN,0.863978,Administration communale de Val-de-RuzForstwar...,0.874443
4,12905109,26220;24210,The Straumann Group is internationally success...,NaN,NaN,NaN,NaN,NaN,NaN,0.777023,Le groupe Straumann connaît un succès internat...,0.806132
...,...,...,...,...,...,...,...,...,...,...,...,...
95,13469747,62100,Commune de Lausanne Service des parcs et domai...,NaN,NaN,NaN,NaN,NaN,NaN,0.763688,Commune de Lausanne - Service des parcs et dom...,0.848250
96,13494906,24210,Quality Assurance Engineer Bern Switzerland Yo...,NaN,NaN,NaN,NaN,NaN,NaN,0.886732,"Quality Assurance Engineer\n\n\nBern, Switzerl...",0.905191
97,13517245,21420;31180,Solution-oriented creative and dynamic We are ...,NaN,NaN,NaN,NaN,NaN,NaN,0.858123,"Lösungsorientiert, kreativ und dynamisch: Wir ...",0.913560
98,13461980,21620,Green BlueGarden garden and landscaping EFZLau...,NaN,NaN,NaN,NaN,NaN,NaN,0.857883,Grün & BlauGärtner/in (Garten- und Landschafts...,0.916243


In [36]:
topResultsDf["BEST_SKILL_MATCH_1SENT"] = topResultsDf.apply(bestMatchReturn, args=(skillsResultsDf, skillsDf, "LOWEST_DISTANCE", "SKILL_CONTENT"), axis=1)
topResultsDf["BEST_SKILL_MATCH_3SENT"] = topResultsDf.apply(bestMatchReturn, args=(skillsResultsDf, skillsDf, "AVERAGE_DISTANCE_3SENT", "SKILL_CONTENT"), axis=1)
topResultsDf["BEST_SKILL_ID_1SENT"] = topResultsDf.apply(bestMatchReturn, args=(skillsResultsDf, skillsDf, "LOWEST_DISTANCE", "SKILL_ID"), axis=1)
topResultsDf["BEST_SKILL_ID_3SENT"] = topResultsDf.apply(bestMatchReturn, args=(skillsResultsDf, skillsDf, "AVERAGE_DISTANCE_3SENT", "SKILL_ID"), axis=1)

In [37]:
topResultsDf["TOP_SKILL_IDS"] = topResultsDf.apply(bestNMatchReturn, args=(skillsResultsDf, skillsDf, "AVERAGE_DISTANCE_3SENT", "SKILL_ID", 10), axis=1)
topResultsDf["TOP_SKILL_MATCHES"] = topResultsDf.apply(bestNMatchReturn, args=(skillsResultsDf, skillsDf, "AVERAGE_DISTANCE_3SENT", "SKILL_CONTENT", 10), axis=1)
topResultsDf

,JOB_ID,ISCO,JOB_CONTENT,TOP_SKILL_MATCHES,TOP_SKILL_IDS,BEST_SKILL_MATCH_3SENT,BEST_SKILL_ID_3SENT,BEST_SKILL_MATCH_1SENT,BEST_SKILL_ID_1SENT,LOWEST_DISTANCE,ORIGINAL_CONTENT,AVERAGE_DISTANCE_3SENT
0,12220025,21420,We are a successful and independent engineerin...,"[Knowledge of materials, methods, and the tool...","[369, 374, 313, 338, 358, 351, 367, 381, 303, ...","Knowledge of materials, methods, and the tools...",369,"Job requires maintaining composure, keeping em...",363,0.829701,<p>Wir sind ein erfolgreiches und unabhängiges...,0.840660
1,10814567,21420,We are a renowned and successful engineering c...,"[Knowledge of materials, methods, and the tool...","[369, 378, 374, 382, 355, 370, 326, 351, 392, ...","Knowledge of materials, methods, and the tools...",369,"Knowledge of materials, methods, and the tools...",369,0.857967,Wir sind eine renommierte und erfolgreiche Ing...,0.891139
2,12401674,21420,Bill Weyermann Partner AG has been providing e...,[Job requires developing one's own ways of doi...,"[357, 350, 369, 353, 361, 364, 358, 341, 374, ...",Job requires developing one's own ways of doin...,357,"Knowledge of materials, methods, and the tools...",369,0.783709,Bill Weyermann Partner AG erbringt seit über\n...,0.852414
3,12473085,62100,Administration communale de ValdeRuzForstwarti...,"[Diploma equivalent , Diploma or cfc, Email so...","[390, 391, 329, 325, 304, 363, 392, 364, 346, ...",Diploma equivalent,390,Diploma equivalent,390,0.863978,Administration communale de Val-de-RuzForstwar...,0.874443
4,12905109,26220;24210,The Straumann Group is internationally success...,[Knowledge of the structure and content of a f...,"[377, 305, 296, 311, 375, 365, 350, 338, 339, ...",Knowledge of the structure and content of a fo...,377,Microsoft Project or Microsoft Share Point,331,0.777023,Le groupe Straumann connaît un succès internat...,0.806132
...,...,...,...,...,...,...,...,...,...,...,...,...
95,13469747,62100,Commune de Lausanne Service des parcs et domai...,"[Job requires maintaining composure, keeping e...","[363, 304, 329, 325, 364, 391, 392, 346, 354, ...","Job requires maintaining composure, keeping em...",363,Knowledge of the structure and content of a fo...,377,0.763688,Commune de Lausanne - Service des parcs et dom...,0.848250
96,13494906,24210,Quality Assurance Engineer Bern Switzerland Yo...,"[Master’s degree, How important is it to work ...","[392, 341, 374, 323, 351, 329, 372, 300, 383, ...",Master’s degree,392,Master’s degree,392,0.886732,"Quality Assurance Engineer\n\n\nBern, Switzerl...",0.905191
97,13517245,21420;31180,Solution-oriented creative and dynamic We are ...,"[ Job requires being reliable, responsible, an...","[356, 351, 364, 355, 369, 348, 353, 365, 300, ...","Job requires being reliable, responsible, and...",356,Job requires being pleasant with others on the...,355,0.858123,"Lösungsorientiert, kreativ und dynamisch: Wir ...",0.913560
98,13461980,21620,Green BlueGarden garden and landscaping EFZLau...,"[Diploma equivalent , Master’s degree, Diploma...","[390, 392, 391, 329, 354, 363, 325, 359, 355, ...",Diploma equivalent,390,Diploma equivalent,390,0.857883,Grün & BlauGärtner/in (Garten- und Landschafts...,0.916243


In [38]:
annotatedJobs200 = pd.read_csv("Data/AnnotatedJobs200.csv", sep=";")

In [39]:
topResultsDf["GREEN_IDENTITY"] = annotatedJobs200["GREEN_IDENTITY"].astype('bool')

In [40]:
topResultsDf = topResultsDf.sort_values(axis=0, by="AVERAGE_DISTANCE_3SENT")

In [41]:
topResultsDf 

,JOB_ID,ISCO,JOB_CONTENT,TOP_SKILL_MATCHES,TOP_SKILL_IDS,BEST_SKILL_MATCH_3SENT,BEST_SKILL_ID_3SENT,BEST_SKILL_MATCH_1SENT,BEST_SKILL_ID_1SENT,LOWEST_DISTANCE,ORIGINAL_CONTENT,AVERAGE_DISTANCE_3SENT,GREEN_IDENTITY
60,14083430,24210,Loopdor leader in the design and manufacture o...,"[Knowledge of raw materials, production proces...","[383, 357, 300, 350, 367, 370, 392, 364, 356, ...","Knowledge of raw materials, production process...",383,"Motivating, developing, and directing people a...",300,0.598410,"Boucledor, leader dans la conception et la fab...",0.698579,False
65,13227820,21430,WAGO is one of the world's market leaders in s...,[Knowledge of the practical application of eng...,"[374, 329, 392, 383, 368, 324, 372, 390, 376, ...",Knowledge of the practical application of engi...,374,Knowledge of the practical application of engi...,374,0.692059,WAGO gehört weltweit zu den Marktführern von S...,0.763964,True
16,12763960,24210,Intern Quality Assurance wm 100 Job offer numb...,"[Email software, Diploma equivalent , Knowledg...","[329, 390, 383, 391, 377, 356, 355, 375, 367, ...",Email software,329,Understanding written sentences and paragraph...,309,0.696713,Praktikant Quality Assurance (w/m) 100%(\n\nSt...,0.766652,False
33,12047533,31230;12190;21620,Bauführer Gartenbautechniker HF Aemmer AG has ...,[Job requires being pleasant with others on th...,"[355, 351, 365, 354, 356, 300, 341, 358, 392, ...",Job requires being pleasant with others on the...,355,Job requires being pleasant with others on the...,355,0.715953,Bauführer / Gartenbautechniker HF\n\nDie Aemme...,0.773319,False
50,13905590,24210,GF Machining Solutions Christine Sauvain 8 rue...,"[HEC-RAS or IWR-PLAN, Diploma equivalent , Mas...","[325, 390, 392, 395, 329, 391, 355, 364, 356, ...",HEC-RAS or IWR-PLAN,325,HEC-RAS or IWR-PLAN,325,0.719905,GF Machining Solutions\n\n\nMadame Christine S...,0.773502,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,13454254,61300,Fleury ClaudeAgriculturalist EFZCourcelon JU L...,"[Master’s degree, Diploma or cfc, Diploma equi...","[392, 391, 390, 363, 325, 326, 329, 351, 372, ...",Master’s degree,392,Master’s degree,392,0.901568,Fleury ClaudeLandwirt/in EFZCourcelon (JU)\n\n...,0.955302,False
84,13466140,21620,Zysset Partner AGGärtnerin Garten und Landscha...,"[Master’s degree, Job requires being sensitiv...","[392, 354, 363, 391, 304, 355, 364, 351, 356, ...",Master’s degree,392,Diploma equivalent,390,0.948213,Zysset + Partner AGGärtner/in (Garten- und Lan...,0.964070,False
91,13463812,21620,Meyer Gärten AGGärtnerin Garten und Landschaft...,"[Diploma equivalent , Master’s degree, Diploma...","[390, 392, 391, 354, 325, 329, 375, 395, 326, ...",Diploma equivalent,390,Diploma equivalent,390,0.948463,Meyer Gärten AGGärtner/in (Garten- und Landsch...,0.964135,False
27,12919468,21650,Schnupperlehre als Geomatikerin AllgemeinesSch...,[Job requires preferring to work with others r...,"[364, 357, 300, 306, 355, 392, 315, 361, 351, ...",Job requires preferring to work with others ra...,364,Job requires developing one's own ways of doin...,357,0.971396,Schnupperlehre als Geomatiker/in\n\nAllgemeine...,0.974113,False


In [42]:
topResultsDf.to_csv("PrelimSkillsResults200V26.csv")

In [43]:
import matplotlib.pyplot as plt